### Base Model : Transfer Learning based on Inception V3
> Inception V3 모델의 객체 생성, ImageNet dataset으로 pre-trained된 weight 사용


* include_top = False : 분류기는 pre-train에서 제외


In [ ]:
# Base Model
# Inception V3 불러오기 - ImageNet 데이터셋으로 pre-train된 가중치 사용
base_model = applications.InceptionV3(weights='imagenet', include_top=False,input_shape=(ROWS, COLS,3)) # include_top=False : 분류기는 pre-train에서 제외

# 처음부터 150번째 layer까지는 학습이 되지 않도록 설정
for i in base_model.layers[0:150]:
    i.trainable = False

# 150번째 이후 layer들은 학습이 가능하도록 설정
for i in base_model.layers[150::]:
    i.trainable = True

add_model = Sequential()
add_model.add(base_model) # Feature Extraction 부분 추가
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(12, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy'])


87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
print(len(base_model.layers))

311


In [ ]:
# 훈련 데이터의 각 {class, wieght}
train_classes = train_generator.classes
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes
                                    )
class_weights = dict(zip(np.unique(train_classes), class_weights))
class_weights

{0: 1.2473290598290598,
 1: 1.8227946916471507,
 2: 0.8040633608815427,
 3: 0.2782743415564295,
 4: 7.783333333333333,
 5: 0.7846102150537635,
 6: 1.018760907504363,
 7: 11.974358974358974,
 8: 0.7260572139303483,
 9: 1.0676726108824874,
 10: 4.605522682445759,
 11: 1.229594523433386}

>  Training
* checkpoint : 훈련 중, 검증 정확도가 최대가 되는 모델의 가중치 저장한다.


In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='base_model_inception', # 저장할 파일의 경로
    save_weights_only=False, # 전체 모델 저장
    monitor='val_accuracy',
    mode='max', # 검증 정확도의 최대값을 찾는다.
    save_best_only=True, # 최적의 가중치만 저장
    verbose =1) # 진행 상황 출력

history = model.fit(train_generator, validation_data=valid_generator, callbacks=[early_stopping, checkpoint],
                    class_weight=class_weights,
                    epochs=50)

In [ ]:
# 약 4시간 걸림
# checkpoint 경로 다시 살펴보기

In [ ]:
# 재학습 시, 이전에 저장된 모델 가중치를 불러온다.
model.load_weights('base_model_inception')

# 이어서 모델 학습
model.fit(train_generator, validation_data=valid_generator, callbacks=[early_stopping, checkpoint],
                    class_weight=class_weights,
                    epochs=50)

In [ ]:
model.evaluate(valid_generator)

32/37 [========================>.....] - ETA: 2s - loss: 0.9744 - accuracy: 0.8130

In [ ]:
model.save('base_model.inception')

class_weight를 고려한 결과로는, 손실은 0.2정도 줄었지만, accuracy도 0.1정도 줄었다.
> * 클래스 가중치를 통해 불균형한 클래스 분포를 고려하였기 때문에 손실이 줄어들었다.
* 반면, 정확도의 감소는 클래스 가중치를 사용하면서 더 많은 잘못 분류된 샘플이 발생했다.
* 이러한 결과는, 클래스 가중치를 사용하는 것이 모델의 손실을 줄이는 데 도움을 줄 수 있지만, 정확도와의 trade-off 가 발생한 것을 나타낸다.